## Libraries + Env Variables

In [ ]:
!pip install langchain unstructured openai faiss-cpu tiktoken comet comet_llm yfinance

In [ ]:
import seaborn as sns            # plots
import matplotlib.pyplot as plt   # plots
import yfinance as yf         # import Yahoo! Finance

In [ ]:
import pandas as pd
import unstructured
# import streamlit as st
# from streamlit_chat import message
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders.excel import UnstructuredExcelLoader
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
import tempfile

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<enter key>"

## Create Functions for Model

In [ ]:
class SecurityBuddy:

  def __init__(self, tickers, market_comparison=False):
    self.tickers = tickers

    if market_comparison==True:
      self.market_indices = ['^DJI', '^IXIC', '^GSPC'] # Dow Jones, Nasdaq and S&P500
    else:
      self.market_indices = None

    self.symbols = self.get_symbols()

  def get_symbols(self):
    symbols = []
    if isinstance(self.tickers, list):
      tickers = [x.upper() for x in self.tickers]
      tickers.sort()
    else:
      tickers = [self.tickers.upper()]

    if self.market_indices is not None:
        for index in self.market_indices:
          self.tickers.append(index)
        symbols = self.tickers
    else:
        symbols = self.tickers
    return symbols

  def get_security_financial_information(self):

    # parameters to download
    fin_info = ["shortName", "sector", "industry", "quoteType", "exchange", "totalAssets", "marketCap", "beta", "trailingPE", "volume", "averageVolume", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "dividendRate", "phone"]

    #  create dataframe
    yf_info = pd.DataFrame(index = fin_info, columns = self.symbols)

    for i in self.symbols:
      l = []             # initialize
      x = yf.Ticker(i)   # get ticker info
      for j in fin_info:
        if 'date' in j.lower():
          d = pd.to_datetime(x.info[j])
          if d is not None:
            l.append(d.strftime("%Y-%m-%d"))  # format date
        else:
          try:      # some parameters error
            l.append(x.info[j])
          except:   # ignore error and continue
            l.append("")

      yf_info[i] = l

    return yf_info

In [ ]:
## test function
# tickers = ['MSFT', 'GOOGL']
# buddy = SecurityBuddy(tickers, False)
# info = buddy.get_security_financial_information()

In [ ]:
#  SET ticker, yf_period and yf_interval
#  ticker - securities to download
# ticker = ["AAPL", "GOOG"]


#  time period and interval
# yf_period = "5y"   # 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# yf_interval = "1m"    # 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo

## Create Tools

In [ ]:
from typing import Type

from langchain.tools import BaseTool
from pydantic import BaseModel, Field

In [ ]:
class FinancialBuddyInput(BaseModel):
    """Inputs for tool class"""

    tickers: Union[str, list] = Field(description="Ticker symbol(s) of the stock(s)")
    market_comparison: bool = Field(description="boolean value for whether or not to get market information too")


class CurrentFinancialInformationTool(BaseTool):
    name = "get_security_financial_information"
    description = """
        Useful when you want to get high-level financial information about a stock.
        You should enter the stock/index ticker symbols recognized by yahoo finance
        """
    args_schema: Type[BaseModel] = FinancialBuddyInput

    def _run(self, tickers: Union[str, list], market_comparison: bool):
        buddy = SecurityBuddy(tickers, market_comparison)
        response = buddy.get_security_financial_information()
        return response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_security_financial_information does not support async")

## Create Default Agent

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.9)

tools = [CurrentFinancialInformationTool()]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [ ]:
response = agent.run(
    "Which company has a higher market cap? Microsoft or Google?"
    )

In [144]:
response

"Microsoft has a higher market cap compared to Google. Microsoft's market cap is $2,763,704,500,224, while Google's market cap is $1,641,913,122,816."

## WIP - Create Custom Agent

In [ ]:
from langchain.agents import AgentExecutor, BaseSingleActionAgent, Tool
from typing import Any, List, Tuple, Union

from langchain.schema import AgentAction, AgentFinish

In [ ]:
class CustomAgent(BaseSingleActionAgent):
    """Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool=tool, tool_input=kwargs["input"], log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool=tool, tool_input=kwargs["input"], log="")